In [2]:
import numpy as np

import sys
sys.path.append('.')

from sfepy.base.base import output, IndexedStruct, Struct
from sfepy.discrete import (FieldVariable, Material, Integral, Integrals,
                            Equation, Equations, Problem)
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.discrete.fem.geometry_element import geometry_data
from sfepy.terms import Term
from sfepy.discrete.conditions import Conditions, EssentialBC
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
from sfepy.mechanics.matcoefs import stiffness_from_youngpoisson
from sfepy import data_dir


#from sfepy.examples.linear_elasticity.its2D_2 import stress_strain
#from sfepy.solvers.auto_fallback import AutoDirect
from sfepy.examples.linear_elasticity.its2D_3 import nodal_stress


In [76]:
mesh = Mesh.from_file('triangle2d.mesh')
domain = FEDomain('domain', mesh)

omega = domain.create_region('Omega', 'all')
top = domain.create_region('Top', 'vertex 2', 'vertex')
bottom_left = domain.create_region('Bottom_Left', 'vertex 0', 'vertex')                                        
bottom_right = domain.create_region('Bottom_Right', 'vertex 1', 'vertex')

field = Field.from_args('fu', np.float64, 'vector', omega,
                        approx_order=2)

u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

# define elastic properties:
# Aluminum properties in SI units
E = 7.0e10  # Young's Modulus in Pa (70 GPa for Aluminum)
nu = 0.33  # Poisson's ratio for Aluminum
D = stiffness_from_youngpoisson(2, young=E, poisson=nu)

aluminum = Material('Aluminum', D=D)

#set up loading:
# Define forces in Newtons
fy = -6e3  # 1 kn is 1e3 in negative y direction (down).
force_ratio = np.tan(np.radians(10))  # Work with an angle of 10 degrees.
fx = -fy * force_ratio  # Horizontal force in positive x direction
load = Material('Load', values={'.val' : [fx, fy]})

integral = Integral('i', order=2)
integral0 = Integral('i', order=0)

t1 = Term.new('dw_lin_elastic(Aluminum.D, v, u)',
                integral, omega, Aluminum=aluminum, v=v, u=u)
t2 = Term.new('dw_point_load(Load.val, v)',
                integral0, top, Load=load, v=v)
eq = Equation('balance', t1 + t2)
eqs = Equations([eq])

ls = ScipyDirect({})

nls_status = IndexedStruct()
nls = Newton({}, lin_solver=ls, status=nls_status)

pb = Problem('elasticity', equations=eqs)

# Apply boundary conditions to the two bottom vertices
fix_bottom_left = EssentialBC('fix_bottom_left', bottom_left, {'u.all': 0.0})  # Fix both x and y displacements
fix_bottom_right = EssentialBC('fix_bottom_right', bottom_right, {'u.all': 0.0})  # Fix both x and y displacements

# Add the boundary conditions to the problem setup
pb.set_bcs(ebcs=Conditions([fix_bottom_left, fix_bottom_right]))

pb.set_solver(nls)

# Solve the problem.
variables = pb.solve()
output(nls_status)

# Postprocess the solution.
out = variables.create_output()

ev = pb.evaluate
strain = ev('ev_cauchy_strain.2.Omega(u)', mode='el_avg')
stress = ev('ev_cauchy_stress.2.Omega(Aluminum.D, u)', mode='el_avg',
                copy_materials=False)

out['cauchy_strain'] = Struct(name='output_data', mode='cell',
                                  data=strain, dofs=None)
out['cauchy_stress'] = Struct(name='output_data', mode='cell',
                                  data=stress, dofs=None)

pb.save_state('triangle_load.vtk', out=out)

## Calculate nodal stresses.

# Setup
gdata = geometry_data['2_3']
nc = len(gdata.coors)

integral_vn = Integral('ivn', coors=gdata.coors,
                        weights=[gdata.volume / nc] * nc)

integrals = Integrals([integral_vn])

# Update problem time
pb.time_update()

# Evaluate stress at quadrature points
stress1 = pb.evaluate('ev_cauchy_stress.ivn.Omega(Aluminum.D, u)', mode='qp', 
                        integrals=integrals, copy_materials=False)

# Create stress field variable from quadrature points
sfield = Field.from_args('stress1', np.float64, (3,), omega)
svar = FieldVariable('sigma', 'parameter', sfield, 
                        primary_var_name='(set-to-None)')
svar.set_from_qp(stress1, integrals['ivn'])


sfepy: reading mesh (triangle2d.mesh)...
sfepy:   number of vertices: 279
sfepy:   number of cells:
sfepy:     2_3: 487
sfepy: ...done in 0.01 s
sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (2084, 2084)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 45800 (1.05e+00% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     Aluminum
sfepy:     Load
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 6.092560e+03 (rel: 1.000000e+00)
sfepy:   residual:    0.00 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.01 [s]
sfepy: warning: linear system solution precision is lower then the value set in solver options! (err = 7.472693e-10 < 1.000000e-10)
sfepy: nls: iter: 1, residual: 7.488952e-10 (rel: 1.229196e-13)


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy: solved in 1 steps in 0.02 seconds
sfepy: IndexedStruct
  condition:
    1
  err:
    7.488952182009181e-10
  err0:
    6092.5596713144705
  ls_n_iter:
    -1
  n_iter:
    1
  time:
    0.010205792001215741
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']
sfepy: equation "tmp":
sfepy: ev_cauchy_strain.2.Omega(u)
sfepy: updating materials...
sfepy: ...done in 0.00 s
sfepy: equation "tmp":
sfepy: ev_cauchy_stress.2.Omega(Aluminum.D, u)
sfepy: updating materials...
sfepy:     Aluminum
sfepy: ...done in 0.00 s


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy: updating variables...
sfepy: ...done
sfepy: equation "tmp":
sfepy: ev_cauchy_stress.ivn.Omega(Aluminum.D, u)
sfepy: updating materials...
sfepy:     Aluminum
sfepy: ...done in 0.00 s


In [4]:
# stress data
sdata = svar()

# Example node indices to print stresses for

specific_nodes = [0, 1, 2]  # Replace these with actual node indices you're interested in

for node in specific_nodes:
    # Horizontal tensile stress (sigma_xx)
    sigma_xx = sdata[node, 0]

    # Vertical compressive stress (sigma_yy)
    sigma_yy = sdata[node, 1]

    # Print stress at the node
    print(f'Node {node}: Horizontal stress (sigma_xx) = {sigma_xx:.5e} Pascals')
    print(f'Node {node}: Vertical stress (sigma_yy) = {sigma_yy:.5e} Pascals')

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [35]:
sfield = Field.from_args('stress', np.float64, (3,), omega)

In [36]:
svar = FieldVariable('sigma', 'parameter', sfield, 
                        primary_var_name='(set-to-None)')


In [ ]:
print(svar)

In [40]:
svar.set_from_qp(stress, integrals['ivn'])

In [ ]:
print(svar)

In [58]:
# Assuming 'svar' is your FieldVariable and 'Omega' is a region in your domain:
stress_at_nodes = svar.get_state_in_region(omega, reshape=True)
print(stress_at_nodes)

[[ 3.59105320e+03  1.19257400e+05  4.30668809e+04]
 [ 1.09830754e+05  2.47122980e+04 -5.02412552e+04]
 [ 9.09505089e+02  3.20571370e+05 -6.09525120e+04]
 ...
 [ 1.37729647e+04  2.53558682e+03 -5.68467343e+03]
 [-1.51793612e+03 -2.97686209e+02 -3.37364547e+03]
 [ 1.50378948e+03 -1.85661594e+03  4.48642684e+02]]


In [59]:
fx, fy

(1057.9618842507898, -6000.0)

In [74]:
s_x, s_y = stress_at_nodes[1, 0:2]
s_x, s_y

(109830.75375176674, 24712.29797843522)

In [75]:
np.rad2deg(np.arctan2(s_y, s_x))

12.680571373186755

In [43]:
stress_at_nodes.shape

(279, 3)

In [5]:
sdata

array([-10110.36410068, -33841.92157777,  19750.44623483, ...,
         -592.58826678,   2430.75331937,  -1128.51074078])

In [6]:
sdata.shape

(837,)

In [27]:
279*3

837

In [ ]:
print(sfield)

In [ ]:
svar = FieldVariable('sigma', 'parameter', sfield, 
                        primary_var_name='(set-to-None)')
print(svar)

In [12]:
stress.shape

(487, 3, 3, 1)

In [17]:
stress[100, :, :, :]

array([[[-6571.11723905],
        [-2529.68584952],
        [ 2679.8697351 ]],

       [[-5498.31167126],
        [  109.82483099],
        [ 2036.11533091]],

       [[-5533.54716454],
        [-2067.55462349],
        [ 2254.78004747]]])

In [18]:
stress2 = pb.evaluate('ev_cauchy_stress.ivn.Omega(Aluminum.D, u)', mode='el_avg', 
                        integrals=integrals, copy_materials=False)

sfepy: equation "tmp":
sfepy: ev_cauchy_stress.ivn.Omega(Aluminum.D, u)
sfepy: updating materials...
sfepy:     Aluminum
sfepy: ...done in 0.00 s


In [19]:
stress2.shape

(487, 1, 3, 1)

In [20]:
stress2[0]

array([[[-1475.260774  ],
        [-4360.82777117],
        [ 2540.78099961]]])

In [21]:
stress[0]

(487, 3, 3, 1)

In [22]:
pb.domain.shape.n_el

487

In [23]:
nc

3

In [25]:
print(gdata)

Struct
  conn:
    list: [0, 1, 2]
  coors:
    list: [[0.0, 0.0], [1.0, 0.0], [0.0, 1.0]]
  edges:
    list: [[0, 1], [1, 2], [2, 0]]
  faces:
    None
  get_grid:
    <function _get_grid_2_3 at 0x16290aa20>
  orientation:
    tuple: (0, (1, 2), 1, 2)
  surface_facet_name:
    1_2
  volume:
    0.5


In [39]:
gdata.coors

[[0.0, 0.0], [1.0, 0.0], [0.0, 1.0]]

In [30]:
pb.domain.regions

[Region:Omega, Region:Top, Region:Bottom_Left, Region:Bottom_Right]

In [32]:
print(pb.domain.regions['Omega'])

Region:Omega
  can:
    tuple: (True, True, True)
  can_cells:
    True
  can_edges:
    True
  can_faces:
    False
  can_vertices:
    True
  cmesh:
    CMesh: n_coor: 279, dim 2, tdim: 2, n_el 487
  definition:
    all
  dim:
    2
  domain:
    FEDomain:domain
  entities:
    list: [array([  0,   1,   2, ..., 276, 277, 278], dtype=uint32), array([  0,   1,   2, ..., 762, 763, 764], dtype=uint32), array([  0,   1,   2, ..., 484, 485, 486], dtype=uint32)]
  extra_options:
    None
  is_empty:
    False
  kind:
    cell
  kind_tdim:
    2
  mirror_maps:
    dict with keys: []
  mirror_regions:
    dict with keys: []
  n_v_max:
    279
  name:
    Omega
  parent:
    None
  parse_def:
    KW_All<>
  shape:
    Struct
  tdim:
    2
  true_kind:
    cell
